ここでは[by @al_log](https://qiita.com/ao_log/items/fe9bd42fd249c2a7ee7a)氏によるサンプルプログラムをもとに，
機械学習手法のいろいろを，IRISデータをもとに試みます．

Pythonのsci-kit learnライブラリを利用すれば，どの方法も10行とかからずに実行することができます．
分類できた，という結果のみに注目するのであれば，ここにあげたような多くの手法を必要とはしません．
重要なのは，データの特性に応じて適切な分類方法が決まってくること，
ある方法で分類できたからといってそれで終わりではないことを知っておくことです．
方法の違いの理解には，数学的準備およびプログラミングスキルがある程度必要になるところを説明抜きに，グラフ表現を通じて分類結果の違いとして観察します．

グラフ表示を豊かにするために，はじめに表示関数を準備し，その実行環境にseabornライブラリを利用しています．

最初にIRISデータセットを準備します．部分的には，クラスター分析のときに扱ったものと変わりません．

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
# 標本の大きさ（サイズ）の確認
print(iris.data.shape)
# 各列ラベル（アヤメの種類）の表示
print(iris.target_names)
# zip関数により表示を縮約しています
for data, target in zip(iris.data[:5], iris.target[:5]):
  print(data, target)

読み込んだデータセットをデータフレームの形式で使えるようにします．

In [ ]:
import pandas as pd

df = pd.DataFrame(iris.data, columns=iris.feature_names)
# データフレーム各列のラベルを設定
df['target'] = iris.target
df.loc[df['target'] == 0, 'target'] = "setosa"
df.loc[df['target'] == 1, 'target'] = "versicolor"
df.loc[df['target'] == 2, 'target'] = "virginica"

# データフレームの始めの数行を表示
df.head()

IRISデータセットの特徴を品種ごとの色分けをし，散布図行列に図示します．対角線には各データごとの分布を図示します．

In [ ]:
# 散布図行列の表示
import seaborn as sns

sns.pairplot(df, hue="target")

機械学習分類手法の特徴を色分けで示しやすくするための，表示関数を準備します．
プログラミング的にはここが最も複雑になっています．

In [4]:
# Xとyの設定
X = iris.data[:, [0, 2]]
y = iris.target

import numpy as np
import matplotlib.pyplot as plt

# グラフの共通設定
h = .02  # Xとyの値に応じて描画範囲を定める
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

def decision_boundary(clf, X, y, ax, title):
    clf.fit(X, y)

    # 値 [x_min, x_max]x[y_min, y_max] の範囲で色の設定
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # 塗りつぶし色の決定
    Z = Z.reshape(xx.shape)
    ax.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

    # 散布図の点を描画
    ax.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Paired)

    # ラベル表示
    ax.set_title(title)
    ax.set_xlabel('sepal length')
    ax.set_ylabel('petal length')

k近傍法で分類します．
k近傍法はよく使われる手法であり，k組に分類します．
何組に分ける（kの値設定）かは自明なことではありません．
以下ではkを4通りに試し，それぞれグラフに描画しています．

In [ ]:
# k-近傍法
from sklearn.neighbors import KNeighborsClassifier

# 近傍数の設定
fig, axes = plt.subplots(1, 4, figsize=(12, 3))

for ax, n_neighbors in zip(axes, [1, 3, 6, 10]):
    title = "%s neighbor(s)"% (n_neighbors)
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    decision_boundary(clf, X, y, ax, title)

ある種の直線で区切られるように分類します．
グラフではその境界が直線になって描画してあります．
定数Cはロジスティク回帰の直線らしきものを定義するパラメータです．

In [ ]:
# ロジスティック回帰，分類境界には直線性が現れる
from sklearn.linear_model import LogisticRegression

fig, axes = plt.subplots(1, 3, figsize=(10, 3))

for ax, C in zip(axes, [0.01, 1, 100]):
    title = "C=%s"% (C)
    clf = LogisticRegression(C=C)
    decision_boundary(clf, X, y, ax, title)

サポートベクトルマシンとは高次元空間の中でベクトルを分類する手法です．
1980年から90年にかけての第2次人工知能ブームにおいて，ニューラルネットワークでは当時の計算機能力において計算しきれない問題をうまく処理するために開発が進みました．
分類境界は直線（超平面）になります．

深層学習（ディープラーニングがある程度実用になった今日においても，データの特徴が合えば，分類の解釈もわかりやすく有用な方法として使われ続けています．


In [ ]:
# サポートベクトルマシーン，点間の余白を大きくする
from sklearn.svm import LinearSVC

fig, axes = plt.subplots(1, 3, figsize=(10, 3))

for ax, C in zip(axes, [0.01, 1, 100]):
    title = "C=%s"% (C)
    clf = LinearSVC(C=C)
    decision_boundary(clf, X, y, ax, title)

カーネル法とは，上の2つの方法が直線境界で分類していたものを曲線境界で分類可能にする方法です．
直線に関するパラメータCに加えて曲線に関するパラメータγ（ガンマ）があります．

In [ ]:
# カーネル法をあわせたサポートベクトルマシーン，境界がカーネル関数を反映した形になる
from sklearn.svm import SVC

fig, axes = plt.subplots(3, 3, figsize=(10, 10))

for ax_row, C in zip(axes, [0.01, 1, 100]):
    for ax, gamma in zip(ax_row, [0.1, 1, 10]):
        title = "C=%s, gamma=%s"% (C, gamma)
        clf = SVC(C=C, gamma=gamma)
        decision_boundary(clf, X, y, ax, title)

決定木による分類です．
クラスター分析で扱ったデンドログラムの作成にもとづく方法です．
平面に図示できる場合，境界は縦横の直線で順次分けて行くように描画されます．

In [ ]:
# 決定木，縦横に直線で次々と切り分けるイメージ
from sklearn.tree import DecisionTreeRegressor

fig, axes = plt.subplots(1, 3, figsize=(10, 3))

for ax, max_depth in zip(axes, [1, 3, 8]):
    title = "max_depth=%s"% (max_depth)
    clf = DecisionTreeRegressor(max_depth=max_depth)
    decision_boundary(clf, X, y, ax, title)

ランダムフォレスト（偶然性によるいくつもの分類を集めたもの）は，二分木で「木（Tree）」と呼んだことに応じて，「森（Forest）＝木を集めたもの」を作り，森から木を分類していくことを指した名前です．



In [ ]:
# ランダムフォレストを用いた決定木，いくつかの分類候補（ランダムに作る）から良いものを選ぶ
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

fig, axes = plt.subplots(1, 2, figsize=(7, 3))
clfs = [RandomForestClassifier(), GradientBoostingClassifier()]
titles = ["RandomForestClassifier", "GradientBoostingClassifier"]

for ax, clf, title in zip(axes, clfs, titles):
    decision_boundary(clf, X, y, ax, title)

ニューラルネットワークによる分類です．
この例題では単に計算時間が多くかかるデメリットばかりです．
問題の性質が上で扱ってきた，ある種の線形性にもとづく分類ができる（上の散布図行列で直感的に分類できそう）問題では効果はありませんが，分類するための情報に欠ける問題では有効性が示されました．

そのことが今日の深層学習の隆盛につながっています．

In [ ]:
# ニューラルネットワーク，データ数が少ないので線形分類とあまり変わらない
from sklearn.neural_network import MLPClassifier

fig, axes = plt.subplots(1, 4, figsize=(12, 3))

for ax, n in zip(axes, [15, 15, 15, 15]):
    title = ""
    clf = MLPClassifier(hidden_layer_sizes=[n, n])
    decision_boundary(clf, X, y, ax, title)

k近傍法では距離の問題に応じて距離などをうまく設定する，分類を進める方法をより適切にとる，などにより有効性が向上します．
いくつ（k）に分類したいという目的がある場合，有効性の高い方法です．
以下のプログラムは最初のk近傍法のバリエーションの一つです．

以下ではkが2から5まで出力しています．

In [ ]:
# k-平均法，点群の重心が離れるようにする
from sklearn.cluster import KMeans

fig, axes = plt.subplots(1, 4, figsize=(12, 3))

for ax, n_clusters in zip(axes, [2, 3, 4, 5]):
    title = "n_clusters=%s"% (n_clusters)
    clf = KMeans(n_clusters=n_clusters)
    decision_boundary(clf, X, y, ax, title)